# Exercise 2 - Get Started with Grounding

In this exercise, we'll start exploring the grounding on SAP AI Core. Grounding allows Large Language Models (LLMs) to reference specific knowledge sources when generating responses, which can help improve accuracy and relevance. 

We'll start with the basics by grounding an LLM using SAP.help as our knowledge source. 

This exercise will cover:

* Basic grounding concepts in SAP AI Core
* How to set up grounding with a sap.help.com
* Comparing responses with and without grounding


## Check your connection to AI Core 
👀  In the ```TECHE2025-167/excercise/init_env.py``` the values from ```TECHED2025-AI167/.aicore-config.json``` are assigned to environmental variables. That way the **SAP Cloud SDK for AI(Python)** will connect to AI Core. 

## Orchestration Service and Resource Group
Before we get into the exercise some information upfront. 
The grounding service we are going to use is available via the **Orchestration Service on GenAI Hub**. The Orchestration service lets you use all the available models with the same codebase. Once orchestration service is deployed you can access all available models simply by changing the model name parameter. Besides grounding you can also use prompt templating, data masking and content filtering capabilities. 

Besides having the orchestration service up and running in your resource group, you also have to make the resource group available to the grounding service.

Good news upfront: We already took care of it!

Let us check whether both is set properly in our environment variables: 

In [1]:
import init_env
import variables
import os

init_env.set_environment_variables()

print(f"Resource Group is set to: {os.getenv('AICORE_RESOURCE_GROUP')}")
print(f"Orchestration Deployment URL is set to: {variables.AICORE_ORCHESTRATION_DEPLOYMENT_URL}")

Deployment URL extraction response data: {'count': 3, 'resources': [{'id': 'd8b26e53441b6143', 'createdAt': '2026-01-09T18:50:38Z', 'modifiedAt': '2026-01-09T18:50:38Z', 'status': 'RUNNING', 'details': {'resources': {'backendDetails': {'model': {'name': 'gemini-2.5-flash', 'version': '001'}}, 'backend_details': {'model': {'name': 'gemini-2.5-flash', 'version': '001'}}}, 'scaling': {'backendDetails': {}, 'backend_details': {}}}, 'scenarioId': 'foundation-models', 'configurationId': '2757402c-d65b-4d6d-9fd8-3a7f8e300d19', 'latestRunningConfigurationId': '2757402c-d65b-4d6d-9fd8-3a7f8e300d19', 'lastOperation': 'CREATE', 'targetStatus': 'RUNNING', 'submissionTime': '2026-01-09T18:50:52Z', 'startTime': '2026-01-09T18:51:57Z', 'configurationName': 'gemini-2.5-flash_autogenerated', 'deploymentUrl': 'https://api.ai.prod.us-east-1.aws.ml.hana.ondemand.com/v2/inference/deployments/d8b26e53441b6143'}, {'id': 'da9d1c13b14cc370', 'createdAt': '2026-01-08T21:11:13Z', 'modifiedAt': '2026-01-08T21:11:

## Let's start with a simple prompt

To understand the general idea about grounding a model, let us first start with the simple prompt without grounding it. 

### Import the packages we want to use 

In [2]:
from gen_ai_hub.orchestration.models.llm import LLM
from gen_ai_hub.orchestration.models.config import OrchestrationConfig
from gen_ai_hub.orchestration.models.template import Template, TemplateValue
from gen_ai_hub.orchestration.models.message import SystemMessage, UserMessage
from gen_ai_hub.orchestration.service import OrchestrationService

### Assign the model we want to use 

In [3]:
llm = LLM(
    name="gemini-2.5-flash",
    parameters={
        'temperature': 0.0,
    }
)
print("LLM initialized:", llm.name)

LLM initialized: gemini-2.5-flash


### Create a prompt Template
The parameter user_query in the code snippet below is going to hold the user query that you will add later on. 

In [4]:
template = Template(
            messages=[
                SystemMessage("You are a helpful assistant."),
                UserMessage("""Answer the request by providing relevant answers that fit to the request.
                Request: {{ ?user_query }}
                """),
            ]
        )

### Create an orchestration configuration
Next you need to create the orchestration configuration by including the LLM we referenced and the prompt template we just created.

In [5]:
config = OrchestrationConfig(
    template=template,
    llm=llm,
)
print("OrchestrationConfig", config.llm.name)

OrchestrationConfig gemini-2.5-flash


### Execute the query
This configuration we now add to the OrchestrationService and then we run to retrieve the answer. 

In [6]:
import importlib
variables = importlib.reload(variables)

orchestration_service = OrchestrationService(
    api_url=variables.AICORE_ORCHESTRATION_DEPLOYMENT_URL,
    config=config,
)
print("OrchestrationService initialized with API URL:", orchestration_service.api_url, config.llm.name)
result = orchestration_service.run(
    template_values=[
        TemplateValue(
            name="user_query",
            value="What is Joule?"
        )
    ]
)
print(result.orchestration_result.choices[0].message.content)

Deployment URL extraction response data: {'count': 3, 'resources': [{'id': 'd8b26e53441b6143', 'createdAt': '2026-01-09T18:50:38Z', 'modifiedAt': '2026-01-09T18:50:38Z', 'status': 'RUNNING', 'details': {'resources': {'backendDetails': {'model': {'name': 'gemini-2.5-flash', 'version': '001'}}, 'backend_details': {'model': {'name': 'gemini-2.5-flash', 'version': '001'}}}, 'scaling': {'backendDetails': {}, 'backend_details': {}}}, 'scenarioId': 'foundation-models', 'configurationId': '2757402c-d65b-4d6d-9fd8-3a7f8e300d19', 'latestRunningConfigurationId': '2757402c-d65b-4d6d-9fd8-3a7f8e300d19', 'lastOperation': 'CREATE', 'targetStatus': 'RUNNING', 'submissionTime': '2026-01-09T18:50:52Z', 'startTime': '2026-01-09T18:51:57Z', 'configurationName': 'gemini-2.5-flash_autogenerated', 'deploymentUrl': 'https://api.ai.prod.us-east-1.aws.ml.hana.ondemand.com/v2/inference/deployments/d8b26e53441b6143'}, {'id': 'da9d1c13b14cc370', 'createdAt': '2026-01-08T21:11:13Z', 'modifiedAt': '2026-01-08T21:11:

### Looks correct, but not for our context

The model’s answer to “What is Joule?” is *technically* correct because, without any extra context, it defaults to the statistically most common public meaning (the physics unit of energy). Our expectation at SAP TechEd was an SAP‑specific interpretation (e.g., an SAP capability or product named “Joule”), but the prompt contained no SAP signals, product names, or retrieved SAP documents to steer the model. With no domain clues, the generic global prior wins.  

Grounding can solve this: attach a grounding/data-retrieval step (SAP Help, internal docs, knowledge base) and inject them into the prompt (or pipeline) before generation. 
In short: no context → generic answer; grounded context → domain‑specific answer.

💪  Let's do this. 

## Ground your prompt with SAP Help

To add context to the retrieval step we need to some change to our original steps. 

Starting at the template by adding to the UserMessage ```Context:{{ ?grounding_response }}```. Whereas **grounding_response** is the context retrieved from the context information, in this case sap.help.com.


In [7]:

template = Template(
            messages=[
                SystemMessage("You are a helpful assistant."),
                UserMessage("""Answer the request by providing relevant answers that fit to the request.
                Request: {{ ?user_query }}
                Context:{{ ?grounding_response }}
                """),
            ]
        )

### Import the packages we want to use 

In [8]:

from gen_ai_hub.orchestration.models.document_grounding import DocumentGroundingFilter
from gen_ai_hub.orchestration.models.document_grounding import GroundingModule
from gen_ai_hub.orchestration.models.document_grounding import GroundingType
from gen_ai_hub.orchestration.models.document_grounding import DocumentGrounding
from gen_ai_hub.orchestration.models.document_grounding import DataRepositoryType

### Define data repository 
We need to configure the Grounding Module, where we first define the data repository that we want to use via the **filter** parameter. 


In [9]:

filters = [
            DocumentGroundingFilter(    id="SAPHelp",
                                        data_repository_type=DataRepositoryType.URL.value)
        ]


### Create Grounding Configuration
Next we create the grounding configuration by using **GroundingModule** for managing and applying grounding configurations:  
- **type**: "document_grounding_service"
- **config**:  Configuration dictionary including parameter defined in the template and filter that includes the data repository type

In [10]:

grounding_config = GroundingModule(
            type=GroundingType.DOCUMENT_GROUNDING_SERVICE.value,
            config=DocumentGrounding(input_params=["user_query"], output_param="grounding_response", filters=filters)
        )

### Create Orchestration Configuration 

Grounding configuration ```grounding_config``` is now an additional parameter that we add to the Orchestration configuration. 

In [11]:

config = OrchestrationConfig(
    template=template,
    llm=llm,
    grounding=grounding_config
)

### Execute the  Query
Configuration will be added again to the OrchestrationService and then we run to retrieve the answer.

In [12]:
import importlib
variables = importlib.reload(variables)

orchestration_service = OrchestrationService(
    api_url=variables.AICORE_ORCHESTRATION_DEPLOYMENT_URL,
    config=config
)

response = orchestration_service.run(
    template_values=[
        TemplateValue(
            name="user_query",
            value="What is Joule?"
        )
    ]
)

print(response.orchestration_result.choices[0].message.content)

Deployment URL extraction response data: {'count': 3, 'resources': [{'id': 'd8b26e53441b6143', 'createdAt': '2026-01-09T18:50:38Z', 'modifiedAt': '2026-01-09T18:50:38Z', 'status': 'RUNNING', 'details': {'resources': {'backendDetails': {'model': {'name': 'gemini-2.5-flash', 'version': '001'}}, 'backend_details': {'model': {'name': 'gemini-2.5-flash', 'version': '001'}}}, 'scaling': {'backendDetails': {}, 'backend_details': {}}}, 'scenarioId': 'foundation-models', 'configurationId': '2757402c-d65b-4d6d-9fd8-3a7f8e300d19', 'latestRunningConfigurationId': '2757402c-d65b-4d6d-9fd8-3a7f8e300d19', 'lastOperation': 'CREATE', 'targetStatus': 'RUNNING', 'submissionTime': '2026-01-09T18:50:52Z', 'startTime': '2026-01-09T18:51:57Z', 'configurationName': 'gemini-2.5-flash_autogenerated', 'deploymentUrl': 'https://api.ai.prod.us-east-1.aws.ml.hana.ondemand.com/v2/inference/deployments/d8b26e53441b6143'}, {'id': 'da9d1c13b14cc370', 'createdAt': '2026-01-08T21:11:13Z', 'modifiedAt': '2026-01-08T21:11:

### The right context
Grounding succeeded: with help.sap.com context injected, the model produced the correct SAP‑specific answer.

## Summary 
You learned the basic grounding concepts in SAP AI Core and how you to use it improve the retrieval.

In the next exercise you will learn on how to ingest custom documents. Therefore we go back to github repo. 

Continue to - [Exercise 3: Ground your LLM with custom documents](https://github.com/SAP-samples/teched2025-AI167/blob/main/exercises/ex3-1-grounding-vector-api.md)
